In [ ]:
import random
import math

In [ ]:
def read_tsp(filename):
    coords = []
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) == 3 and parts[0].isdigit():
                x, y = float(parts[1]), float(parts[2])
                coords.append((x, y))
    if not coords:
        raise ValueError("File .tsp không hợp lệ: không đọc được tọa độ")

    n = len(coords)
    dist_matrix = [[0.0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if i != j:
                xi, yi = coords[i]
                xj, yj = coords[j]
                dist_matrix[i][j] = round(math.hypot(xi - xj, yi - yj))
    return dist_matrix

In [ ]:
def read_knapsack(prefix):
    with open(f"/Users/Prj TTTH/EC-lab/dataset/knapsack/{prefix}_c.txt", "r") as f:
        capacity = int(f.readline().strip())
    with open(f"/Users/Prj TTTH/EC-lab/dataset/knapsack/{prefix}_p.txt", "r") as f:
        values = [int(x) for x in f.read().split()]
    with open(f"/Users/Prj TTTH/EC-lab/dataset/knapsack/{prefix}_w.txt", "r") as f:
        weights = [int(x) for x in f.read().split()]
    return weights, values, capacity

In [ ]:
class Individual:
    def __init__(self, genome, skill_factor=None):
        self.genome = genome
        self.skill_factor = skill_factor
        self.fitness = None
    def copy(self):
        ind = Individual(self.genome.copy(), self.skill_factor)
        ind.fitness = self.fitness
        return ind


In [ ]:
class TSP:
    def __init__(self, distance_matrix):
        self.distance_matrix = distance_matrix
        self.num_cities = len(distance_matrix)
    def decode(self, shared_genome):
        sorted_indices = sorted(range(len(shared_genome)),
                                key=lambda i: shared_genome[i])
        return sorted_indices[:self.num_cities]
    def evaluate(self, individual):
        genome = self.decode(individual.genome)
        dist = 0
        for i in range(self.num_cities):
            dist += self.distance_matrix[genome[i]][genome[(i+1) % self.num_cities]]
        individual.fitness = -dist
        return individual
    def generate_genome(self, L=51):
        return [random.random() for _ in range(L)]

In [ ]:
class Knapsack:
    def __init__(self, weights, values, capacity):
        self.weights = weights
        self.values = values
        self.capacity = capacity
        self.num_items = len(weights)
    def decode(self, shared_genome):
        return [1 if g >= 0.5 else 0 for g in shared_genome[:self.num_items]]
    def evaluate(self, individual):
        genome = self.decode(individual.genome)
        best_val = -1e9
        n = len(genome)
        for start in range(0, max(0, n - 15 + 1)):
            sub = genome[start:start+15]
            total_w = 0
            total_v = 0
            for i, g in enumerate(sub, start=start):
                if g == 1:
                    total_w += self.weights[i]
                    total_v += self.values[i]
            if total_w <= self.capacity:
                best_val = max(best_val, total_v)
        if best_val == -1e9:
            best_val = -1e9
        individual.fitness = best_val
        return individual
    def generate_genome(self, L=51):
        return [random.random() for _ in range(L)]

In [ ]:
def tsp_crossover(p1, p2):
    n = len(p1)
    c1, c2 = p1.copy(), p2.copy()
    i, j = sorted(random.sample(range(n), 2))
    c1[i:j], c2[i:j] = p2[i:j], p1[i:j]
    return c1, c2

In [ ]:
def knapsack_crossover(p1, p2):
    point = random.randint(1, len(p1)-1)
    return p1[:point] + p2[point:], p2[:point] + p1[point:]

In [ ]:
def cross_task_crossover(p1, p2, alpha=0.5):
    c1, c2 = [], []
    for x, y in zip(p1, p2):
        low = min(x, y) - alpha * abs(x-y)
        high = max(x, y) + alpha * abs(x-y)
        c1.append(random.uniform(low, high))
        c2.append(random.uniform(low, high))
    return c1, c2

In [ ]:
def tsp_mutation(g):
    i, j = random.sample(range(len(g)), 2)
    g[i], g[j] = g[j], g[i]
    return g

In [ ]:
def knapsack_mutation(g):
    i = random.randint(0, len(g)-1)
    g[i] = 1.0 - g[i]
    return g

In [ ]:
class MFEA:
    def __init__(self, problems, pop_size=100, n_gen=500,
                 crossover_rate=0.9, mutation_rate=0.1, rmp=0.3):
        self.problems = problems
        self.pop_size = pop_size
        self.n_gen = n_gen
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.rmp = rmp
        self.population = []

    def initialize(self):
        self.population = []
        ratio = random.uniform(0.4, 0.6)
        n0 = int(self.pop_size * ratio)
        n1 = self.pop_size - n0
        for _ in range(n0):
            g = self.problems[0].generate_genome()
            self.population.append(Individual(g, 0))
        for _ in range(n1):
            g = self.problems[1].generate_genome()
            self.population.append(Individual(g, 1))
        random.shuffle(self.population)

    def evaluate(self):
        for ind in self.population:
            self.problems[ind.skill_factor].evaluate(ind)

    def run(self, return_pop=False):
        self.initialize()
        self.evaluate()

        # compute initial best per task
        best_by_skill = {0: None, 1: None}
        for ind in self.population:
            b = best_by_skill[ind.skill_factor]
            if b is None or (ind.skill_factor == 0 and ind.fitness < b.fitness) or (ind.skill_factor == 1 and ind.fitness > b.fitness):
                best_by_skill[ind.skill_factor] = ind.copy()

        # history: list of tuples (best_tsp, best_knap) per generation (use None if missing)
        best_history = []
        best_history.append((best_by_skill[0].fitness if best_by_skill[0] else None,
                             best_by_skill[1].fitness if best_by_skill[1] else None))

        for gen in range(self.n_gen):
            p1, p2 = random.sample(self.population, 2)

            children = []
            if p1.skill_factor == p2.skill_factor:
                if random.random() < self.crossover_rate:
                    if p1.skill_factor == 0:
                        c1g, c2g = tsp_crossover(p1.genome, p2.genome)
                    else:
                        c1g, c2g = knapsack_crossover(p1.genome, p2.genome)
                else:
                    c1g, c2g = p1.genome.copy(), p2.genome.copy()
                if random.random() < self.mutation_rate:
                    if p1.skill_factor == 0:
                        c1g, c2g = tsp_mutation(c1g), tsp_mutation(c2g)
                    else:
                        c1g, c2g = knapsack_mutation(c1g), knapsack_mutation(c2g)
                children = [Individual(c1g, p1.skill_factor),
                            Individual(c2g, p1.skill_factor)]
            else:
                if random.random() < self.rmp:
                    c1g, c2g = cross_task_crossover(p1.genome, p2.genome)
                    children = [Individual(c1g, p1.skill_factor),
                                Individual(c2g, p2.skill_factor)]
                else:
                    children = [p1.copy(), p2.copy()]

            # add children and evaluate
            self.population.extend(children)
            self.evaluate()

            # split by skill
            group0 = [ind for ind in self.population if ind.skill_factor == 0]
            group1 = [ind for ind in self.population if ind.skill_factor == 1]
            group0.sort(key=lambda ind: ind.fitness, reverse=True)
            group1.sort(key=lambda ind: ind.fitness, reverse=True)

            ratio = random.uniform(0.4, 0.6)
            n0 = int(self.pop_size * ratio)
            n1 = self.pop_size - n0
            next_pop = []
            next_pop.extend([ind.copy() for ind in group0[:n0]])
            next_pop.extend([ind.copy() for ind in group1[:n1]])

            # if due to shortage of candidates total < pop_size, fill with best available from both groups
            if len(next_pop) < self.pop_size:
                leftover = (group0[n0:] if len(group0) > n0 else []) + (group1[n1:] if len(group1) > n1 else [])
                k = 0
                while len(next_pop) < self.pop_size and k < len(leftover):
                    next_pop.append(leftover[k].copy())
                    k += 1

            self.population = next_pop
            random.shuffle(self.population)

            for ind in self.population:
                b = best_by_skill[ind.skill_factor]
                if b is None:
                    best_by_skill[ind.skill_factor] = ind.copy()
                else:
                    if ind.fitness > b.fitness:
                        best_by_skill[ind.skill_factor] = ind.copy()

            best_history.append((best_by_skill[0].fitness if best_by_skill[0] else None,
                                 best_by_skill[1].fitness if best_by_skill[1] else None))

        if return_pop:
            return best_by_skill, best_history, self.population
        return best_by_skill, best_history

In [ ]:
class Experiment:
    def __init__(self, algo, problems, seed_list=range(30), **algo_kwargs):
        self.algo = algo
        self.problems = problems if isinstance(problems, list) else [problems]
        self.seeds = seed_list
        self.algo_kwargs = algo_kwargs

    def run(self, verbose=True):
        all_best = []
        all_histories = []

        for i, seed in enumerate(self.seeds):
            random.seed(seed)
            algorithm = self.algo(self.problems, **self.algo_kwargs)
            best, history, population = algorithm.run(return_pop=True)

            best_per_problem = []
            for pid, prob in enumerate(self.problems):
                best_ind = None
                best_fit = -1e18
                for ind in population:
                    cand = ind.copy()
                    prob.evaluate(cand)
                    if cand.fitness > best_fit:
                        best_fit = cand.fitness
                        best_ind = cand
                best_per_problem.append(best_ind)

            all_best.append(best_per_problem)
            all_histories.append(history)

            if verbose:
                for pid, ind in enumerate(best_per_problem):
                    pname = type(self.problems[pid]).__name__
                    fitness_str = ind.fitness if ind is not None else None
                    print(f"[Seed {seed}] Best {pname} fitness = {fitness_str}")

        return all_best, all_histories

In [202]:
if __name__ == "__main__":
    tsp_file = "/Users/Prj TTTH/EC-lab/dataset/tsp/eil51.tsp"
    distance_matrix = read_tsp(tsp_file)
    tsp_problem = TSP(distance_matrix)

    weights, values, capacity = read_knapsack("p07")
    knapsack_problem = Knapsack(weights, values, capacity)

    problems = [tsp_problem, knapsack_problem]

    exp = Experiment(
        MFEA, problems,
        seed_list=range(30),
        pop_size=100, n_gen=500,
        crossover_rate=0.9, mutation_rate=0.2, rmp=0.3
    )

    results, all_histories = exp.run()

    best_tsp = None
  
    best_knap = None

    for best_per_problem in results:
        tsp_ind, knap_ind = best_per_problem

        if tsp_ind is not None and (best_tsp is None or tsp_ind.fitness > best_tsp.fitness):
            best_tsp = tsp_ind.copy()

        if knap_ind is not None and (best_knap is None or knap_ind.fitness > best_knap.fitness):
            best_knap = knap_ind.copy()

    print("\n===== FINAL RESULTS =====")
    if best_tsp:
        print(f"Best TSP fitness (tour length): {best_tsp.fitness}")
        print(f"Genome: {best_tsp.genome}")
    if best_knap:
        print(f"Best Knapsack fitness (value): {best_knap.fitness}")
        print(f"Genome: {best_knap.genome}")


[Seed 0] Best TSP fitness = -1314
[Seed 0] Best Knapsack fitness = 1447
[Seed 1] Best TSP fitness = -1156
[Seed 1] Best Knapsack fitness = 1438
[Seed 2] Best TSP fitness = -1345
[Seed 2] Best Knapsack fitness = 1433
[Seed 3] Best TSP fitness = -1260
[Seed 3] Best Knapsack fitness = 1422
[Seed 4] Best TSP fitness = -1347
[Seed 4] Best Knapsack fitness = 1442
[Seed 5] Best TSP fitness = -1266
[Seed 5] Best Knapsack fitness = 1447
[Seed 6] Best TSP fitness = -1323
[Seed 6] Best Knapsack fitness = 1445
[Seed 7] Best TSP fitness = -1280
[Seed 7] Best Knapsack fitness = 1444
[Seed 8] Best TSP fitness = -1332
[Seed 8] Best Knapsack fitness = 1448
[Seed 9] Best TSP fitness = -1304
[Seed 9] Best Knapsack fitness = 1444
[Seed 10] Best TSP fitness = -1320
[Seed 10] Best Knapsack fitness = 1446
[Seed 11] Best TSP fitness = -1273
[Seed 11] Best Knapsack fitness = 1449
[Seed 12] Best TSP fitness = -1343
[Seed 12] Best Knapsack fitness = 1458
[Seed 13] Best TSP fitness = -1345
[Seed 13] Best Knapsack